<a href="https://colab.research.google.com/github/deivismartinez/DDE/blob/main/Metrics_NER_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=ea37e5de3b8c1f02f9623fdbdde826de7b9d67aa4e869c6ecdef1d2d724ad8ff
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics.sequence_labeling import precision_score, recall_score
from seqeval.metrics import f1_score
import glob
import os
from typing import List
import zipfile
from google.colab import files

In [3]:
!cd /content && git clone https://github.com/nlplab/brat.git
!wget https://github.com/deivismartinez/NER-Medical-Uninorte/raw/main/system2024.zip?download=1 -O system.zip

Cloning into 'brat'...
remote: Enumerating objects: 17077, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17077 (delta 17), reused 40 (delta 15), pack-reused 17029
Receiving objects: 100% (17077/17077), 84.48 MiB | 8.79 MiB/s, done.
Resolving deltas: 100% (10311/10311), done.
--2024-08-01 23:29:01--  https://github.com/deivismartinez/NER-Medical-Uninorte/raw/main/system2024.zip?download=1
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/deivismartinez/NER-Medical-Uninorte/main/system2024.zip [following]
--2024-08-01 23:29:01--  https://raw.githubusercontent.com/deivismartinez/NER-Medical-Uninorte/main/system2024.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.git

In [4]:
!unzip system.zip -d /content/systemd/

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/systemd/system/model_36/test2/S0004-06142007000600014-1.ann  
  inflating: /content/systemd/system/model_36/test2/S0004-06142007000600014-1.txt  
  inflating: /content/systemd/system/model_36/test2/S0004-06142007000600016-2.ann  
  inflating: /content/systemd/system/model_36/test2/S0004-06142007000600016-2.txt  
  inflating: /content/systemd/system/model_36/test2/S0004-06142007000700015-1.ann  
  inflating: /content/systemd/system/model_36/test2/S0004-06142007000700015-1.txt  
  inflating: /content/systemd/system/model_36/test2/S0004-06142007000900010-1.ann  
  inflating: /content/systemd/system/model_36/test2/S0004-06142007000900010-1.txt  
  inflating: /content/systemd/system/model_36/test2/S0004-06142007000900011-1.ann  
  inflating: /content/systemd/system/model_36/test2/S0004-06142007000900011-1.txt  
  inflating: /content/systemd/system/model_36/test2/S0004-06142007000900013-1.ann  
  inflating: /c

In [ ]:
num = 0
gold_files = glob.glob("/content/systemd/system/gold/*.ann")
for file in gold_files:
  print(num)
  num = num + 1

In [6]:
def validate_overlap(start_pred, end_pred, file_name):
  with open(file_name, "rt") as file:
    x = [row for row in file.readlines() if "NOMBRE" in row]
    for linesGold in x:
      if len(linesGold.split("\t")) > 1 and len(linesGold.split("\t")[1].split(" ")) > 1:
        start_gold = int(linesGold.split("\t")[1].split(" ")[1])
        end_gold = int(linesGold.split("\t")[1].split(" ")[2])
        if start_pred == start_gold and end_pred == end_gold:
          return(start_gold,end_gold)
        if start_pred == start_gold and abs(end_pred - end_gold) <= 6:
          return(start_gold,end_gold)
        elif end_pred == end_gold and abs(start_pred - start_gold) <= 6:
          return(start_gold,end_gold)
        elif abs(end_pred - end_gold) <= 1 and abs(start_pred - start_gold) <= 1:
          return(start_gold,end_gold)
  return(start_pred,end_pred)
      #print(f'Start Pred: {start_pred}, End Pred: {end_pred}, Start Gold: {start_gold}, Start Gold: {end_gold}')

def change_by_overlap(file_name_pred, file_name_gold):
  with open(file_name_pred, "rt") as file:
    x = [row for row in file.readlines() if "NOMBRE" in row]
    y = []
    for linesPred in x:
      if len(linesPred.split("\t")) > 1 and len(linesPred.split("\t")[1].split(" ")) > 1:
        start_pred = int(linesPred.split("\t")[1].split(" ")[1])
        end_pred = int(linesPred.split("\t")[1].split(" ")[2])
        new_start, new_end = validate_overlap(start_pred, end_pred, file_name = file_name_gold)
        if start_pred != new_start or end_pred != new_end:
          #print(f'n_s: {new_start}, n_e:{new_end}')
          linesPred = linesPred.replace(' '+str(start_pred)+' ', ' '+str(new_start)+' ')
          linesPred = linesPred.replace(' '+str(end_pred), ' '+str(new_end))
      y.append(linesPred)
    with open(file_name_pred, "wt") as file:
      file.writelines(y)

def change(file_name):
  change_tags(old_value = "NOMBRE_PERSONAL_SANITARIO", new_value="NOMBRE",file_name=file_name)
  change_tags(old_value = "NOMBRE_SUJETO_ASISTENCIA", new_value="NOMBRE",file_name=file_name)
  change_tags(old_value = "FAMILIARES_SUJETO_ASISTENCIA", new_value="NOMBRE",file_name=file_name)
  change_tags(old_value = "OTROS_SUJETO_ASISTENCIA", new_value="NOMBRE",file_name=file_name)

  change_tags(old_value = "TERRITORIO", new_value="LUGAR",file_name=file_name)
  change_tags(old_value = "PAIS", new_value="LUGAR",file_name=file_name)
  change_tags(old_value = "CALLE", new_value="LUGAR",file_name=file_name)

  change_tags(old_value = "CENTRO_SALUD", new_value="ORGANIZACION",file_name=file_name)
  change_tags(old_value = "HOSPITAL", new_value="ORGANIZACION",file_name=file_name)
  change_tags(old_value = "INSTITUCION", new_value="ORGANIZACION",file_name=file_name)

def change_tags(old_value, new_value, file_name):
  with open(file_name, "rt") as file:
    x = file.read()

  with open(file_name, "wt") as file:
    x = x.replace(old_value,new_value)
    file.write(x)

def do_changes():
  gold_files = glob.glob("/content/systemd/system/gold/*.ann")
  for file in gold_files:
    change(file)

  for model_name in glob.glob("/content/systemd/system/*"):
    if "model_" in model_name:
      gold_files = glob.glob(f"{model_name}/test2/*.ann")
      for file in gold_files:
        change(file)

def delete_tags(file_name):
  with open(file_name, "rt") as file:
    x = [row.replace('▁','') for row in file.readlines() if "NOMBRE" in row] # or "LUGAR" in row or "ORGANIZACION" in row]
  with open(file_name, "wt") as file:
    file.writelines(x)

def do_file_gold_conll():
  gold_files = glob.glob("/content/systemd/system/gold/*.txt")

  for file in gold_files:
    delete_tags(file[:-3]+"ann")

    command = f"""
    python /content/brat/tools/anntoconll.py "{file}"
    """
    os.system(command)

def do_file_pred_conll(model):
  pred_files = glob.glob(f"{model}/test2/*.txt")

  for file in pred_files:
    delete_tags(file[:-3]+"ann")
    path = file[:-3]+"ann"
    path = path.replace(model+'/test2/','/content/systemd/system/gold/')
    #print(path)
    change_by_overlap(file[:-3]+"ann",path)
    command = f"""
    python /content/brat/tools/anntoconll.py "{file}"
    """
    os.system(command)


def create_file():
  do_file_gold_conll()
  for model_name in glob.glob("/content/systemd/system/*"):
    if "model_" in model_name:
      do_file_pred_conll(model=model_name)

def read_tags(fpath: str) -> List:
  lines = open(fpath, 'r').read()
  lines = lines.split("\n")
  lines = [line.split("\t") for line in lines]
  tags = [line[0] for line in lines if len(line) > 1]
  return tags

def do_table(model,file_t,file_tp):
  gold_files = glob.glob("/content/systemd/system/gold/*.conll")
  pred_files = glob.glob(f"{model}/test2/*.conll")
  gold_tags = [read_tags(file) for file in gold_files]
  pred_tags = [read_tags(file) for file in pred_files]
  f1 = round(f1_score(y_true=gold_tags, y_pred=pred_tags, mode='strict'),4)
  precision = round(precision_score(y_true=gold_tags, y_pred=pred_tags, mode='strict'),4)
  recall = round(recall_score(y_true=gold_tags, y_pred=pred_tags, mode='strict'),4)
  file_tp.write(models_name[model].strip()+'\t'+str(precision).strip()+'\t'+str(recall).strip()+'\t'+str(f1).strip()+'\n')
  statistics = classification_report(y_true=gold_tags, y_pred=pred_tags)
  do_file(statistics=statistics,file_t=file_t,model=models_name[model])

def do_file(statistics,file_t,model):
  rows_new = statistics.split('\n')
  file_t.write(model+'\n')
  for row_new in rows_new:
    for value in row_new.split('\t'):
      file_t.write(value+'\t')
    file_t.write('\n')

In [7]:
do_changes()

In [8]:
models_name = {
"/content/systemd/system/model_1":"roberta-large-NER",
"/content/systemd/system/model_2":"bert-base-multilingual-cased-ner-spanish",
"/content/systemd/system/model_3":"anonymizer-beto-cased-flair",
"/content/systemd/system/model_4":"flair-ner-spanish-judicial",
"/content/systemd/system/model_5":"wikineural-multilingual-ner",
"/content/systemd/system/model_6":"bertin-base-ner-conll2002-es",
"/content/systemd/system/model_7":"roberta_model_for_anonimization",
"/content/systemd/system/model_8":"roberta-base-bne-capitel-ner-plus",
"/content/systemd/system/model_9":"NER-MEDDOCAN",
"/content/systemd/system/model_10":"bert-base-multilingual-cased-fine_tuned-ner-WikiNeural_Multilingual",
"/content/systemd/system/model_11":"xlm-roberta-large-finetuned-conll03-english",
"/content/systemd/system/model_12":"xlm-roberta-large-finetuned-conll03-german",
"/content/systemd/system/model_13":"ner-multi",
"/content/systemd/system/model_14":"ner-multi-fast",
"/content/systemd/system/model_15":"ner-spanish-large",
"/content/systemd/system/model_16":"AuthorParserModel",
"/content/systemd/system/model_17":"distilbert-base-multilingual-cased-finetuned-conll2003-ner",
"/content/systemd/system/model_18":"xlm-roberta-base-finetuned-panx-ner",
"/content/systemd/system/model_19":"BETO-finetuned-ner-3",
"/content/systemd/system/model_20":"span-marker-bert-base-multilingual-cased-multinerd",
"/content/systemd/system/model_21":"span-marker-bert-base-multilingual-uncased-multinerd",
"/content/systemd/system/model_22":"xlm-roberta-large-ner-spanish",
"/content/systemd/system/model_23":"bert-spanish-cased-finetuned-ner",
"/content/systemd/system/model_24":"RuPERTa-base-finetuned-ner",
"/content/systemd/system/model_25":"TinyBERT-spanish-uncased-finetuned-ner",
"/content/systemd/system/model_26":"NER-fine-tuned-BETO",
"/content/systemd/system/model_27":"flair-ner-multi",
"/content/systemd/system/model_28":"ca_anonimization_core_lg",
"/content/systemd/system/model_29":"es_anonimization_core_lg",
"/content/systemd/system/model_30":"roberta-base-bne-capitel-ner",
"/content/systemd/system/model_31":"roberta-base-bne-capitel-ner-plus",
"/content/systemd/system/model_32":"roberta-large-bne-capitel-ner",
"/content/systemd/system/model_33":"DEBERTA_CIEL",
"/content/systemd/system/model_34":"codeswitch-spaeng-ner-lince",
"/content/systemd/system/model_35":"span-marker-bert-base-conll2002-es",
"/content/systemd/system/model_36":"es_core_news_lg",
"/content/systemd/system/model_37":"es_core_news_md",
"/content/systemd/system/model_38":"es_core_news_sm",
"/content/systemd/system/model_39":"stanza-es",
"/content/systemd/system/model_40":"gliner_multi_pii-v1"
 }

In [9]:
create_file()

In [10]:
file_t= open("tabla_results.txt","w")
file_tp= open("tabla_p.txt","w")
file_tp.write('Modelo\tPrecission\tRecall\tF1\n')
for model_name in glob.glob("/content/systemd/system/*"):
    print(model_name)
    if "model_" in model_name: # and model_name == '/content/systemd/system/model_7':
      try:
        do_table(model=model_name,file_t=file_t,file_tp=file_tp)
      except Exception:
        print("NO se pudo")
        print(Exception)
file_tp.close()
file_t.close()

/content/systemd/system/model_25
/content/systemd/system/model_8
/content/systemd/system/model_20
/content/systemd/system/model_21
/content/systemd/system/model_16
/content/systemd/system/model_12
/content/systemd/system/model_28
/content/systemd/system/model_1
/content/systemd/system/model_34
/content/systemd/system/model_30
/content/systemd/system/model_13
/content/systemd/system/model_15
/content/systemd/system/model_27
/content/systemd/system/model_19
/content/systemd/system/model_10
/content/systemd/system/model_3
/content/systemd/system/model_9
/content/systemd/system/model_26
/content/systemd/system/model_24
/content/systemd/system/model_6
/content/systemd/system/model_11
/content/systemd/system/model_40
/content/systemd/system/model_37
/content/systemd/system/model_38
/content/systemd/system/model_5
/content/systemd/system/model_29
/content/systemd/system/model_23
/content/systemd/system/models.json
/content/systemd/system/model_14
/content/systemd/system/model_36
/content/syst

In [ ]:
f1 = f1_score(y_true=gold_tags, y_pred=pred_tags)
print(f1)

In [ ]:
ii = 0
for name_coll in glob.glob("/content/systemd/systemd/model_7/test2/*"):
  if ".conll" in name_coll:
    ii+=1
print(ii)

In [ ]:
precision = precision_score(y_true=gold_tags, y_pred=pred_tags)
print(precision)

In [37]:
y_true = [['O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
y_pred = [['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
f1_score(y_true, y_pred)

0.5

In [38]:
do_file()

TypeError: do_file() missing 3 required positional arguments: 'statistics', 'file_t', and 'model'

In [ ]:
classification_report(y_true, y_pred)

In [11]:
def comprimir():
  fantasy_zip = zipfile.ZipFile('/content/systemd.zip', 'w')
  for folder, subfolders, filesFolder in os.walk('/content/systemd'):
      for file in filesFolder:
          if file.endswith('.ann') or file.endswith('.txt') or file.endswith('.json') or file.endswith('.conll'):
              fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), '/content/systemd'), compress_type = zipfile.ZIP_DEFLATED)
  fantasy_zip.close()

In [12]:
def downloadFile():
  files.download('/content/systemd.zip')

In [13]:
comprimir()
downloadFile()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>